<a href="https://colab.research.google.com/github/YatinGupta777/Whatsapp-Chat-Analyzer/blob/master/WhatsApp_Chat_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
import re
import pandas as pd
from matplotlib import pyplot as plt
from pandas import ExcelWriter
from google.colab import files
import numpy as np

In [0]:

def startsWithDate(s):

    #MM/DD/YY
    pattern = '^(([0-9])|((1)[0-2]))(\/)([0-9])|([0-2][0-9]|(3)[0-1])(\/)(\d{2}|\d{4})'
    result = re.match(pattern, s)
    if result:
        return True
    return False


def startsWithAuthor(s):
    patterns = [
        '([\w]+):',                        # First Name
        '([\w]+[\s]+[\w]+):',              # First Name + Last Name
        '([\w]+[\s]+[\w]+[\s]+[\w]+):',    # First Name + Middle Name + Last Name
        '([+]\d{2} \d{5} \d{5}):',         # Mobile Number (India)
        '([+]\d{2} \d{3} \d{3} \d{4}):',   # Mobile Number (US)
        '([+]\d{2} \d{4} \d{7})'           # Mobile Number (Europe)
    ]
    pattern = '^' + '|'.join(patterns)
    result = re.match(pattern, s)
    if result:
        return True
    return False

def getDataPoint(line):
    # line = 18/06/17, 22:47 - Loki: Why do you have 2 numbers, Banner?
    
    splitLine = line.split(' - ') # splitLine = ['18/06/17, 22:47', 'Loki: Why do you have 2 numbers, Banner?']
    
    dateTime = splitLine[0] # dateTime = '18/06/17, 22:47'
    
    date, time = dateTime.split(', ') # date = '18/06/17'; time = '22:47'
    
    message = ' '.join(splitLine[1:]) # message = 'Loki: Why do you have 2 numbers, Banner?'
    
    if startsWithAuthor(message): # True
        splitMessage = message.split(': ') # splitMessage = ['Loki', 'Why do you have 2 numbers, Banner?']
        author = splitMessage[0] # author = 'Loki'
        message = ' '.join(splitMessage[1:]) # message = 'Why do you have 2 numbers, Banner?'
    else:
        author = None
    return date, time, author, message

parsedData = [] # List to keep track of data so it can be used by a Pandas dataframe
conversationPath = '/content/drive/My Drive/Colab Notebooks/WhatsApp-Chat-Analyzer/Chat.txt' 
with open(conversationPath, encoding="utf-8") as fp:
    fp.readline() # Skipping first line of the file (usually contains information about end-to-end encryption)
        
    messageBuffer = [] # Buffer to capture intermediate output for multi-line messages
    date, time, author = None, None, None # Intermediate variables to keep track of the current message being processed
    
    while True:
        line = fp.readline() 
        if not line: # Stop reading further if end of file has been reached
            break
        line = line.strip()
       # Guarding against erroneous leading and trailing whitespaces
        if startsWithDate(line): # If a line starts with a Date Time pattern, then this indicates the beginning of a new message
            if len(messageBuffer) > 0: # Check if the message buffer contains characters from previous iterations
                parsedData.append([date, author])#' '.join(messageBuffer)]) # Save the tokens from the previous message in parsedData
            messageBuffer.clear() # Clear the message buffer so that it can be used for the next message
            date, time, author, message = getDataPoint(line) # Identify and extract tokens from the line
            messageBuffer.append(message) # Append message to buffer
        else:
            messageBuffer.append(line) # If a line doesn't start with a Date Time pattern, then it is part of a multi-line message. So, just append to buffer
#pd.set_option('max_rows', 99999)
#pd.set_option('max_colwidth', 400)
#pd.describe_option('max_colwidth')

df = pd.DataFrame(parsedData, columns=['Date', 'Author'])
df_by_date = df.groupby(['Date'],sort=False).size().reset_index()
df_by_date_author = df.groupby(['Author','Date'],sort=False).size().to_frame('Messages').reset_index()
#df_new.sort_values()
#print(df_new.head)
#df.describe()

#df_by_date_author.describe()

#df['Date'].value_counts().plot.barh() # Top 10 Times of the day at which the most number of messages were sent
#plt.xlabel('Number of messages')
#plt.ylabel('Time')



In [155]:
#df_by_date_author.columns['Date','Author','Messages']
#print(type(df_by_date_author))
df_by_date_author.head()

,Author,Date,Messages
0,Pranav,6/10/18,3
1,Daljit,6/10/18,1
2,Aayush,6/10/18,1
3,Pranav,6/11/18,8
4,Daljit,6/11/18,7


In [125]:
authors = df['Author'].unique()
dates = df['Date'].unique()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future
  after removing the cwd from sys.path.


ValueError: ignored

In [158]:
df_structure = pd.DataFrame(index=authors, columns=dates)
df_structure

,6/10/18,6/11/18,6/13/18,6/14/18,6/15/18,6/16/18,6/17/18,6/18/18,6/19/18,6/20/18,6/21/18,6/22/18,6/23/18,6/24/18,6/25/18,6/26/18,6/27/18,6/28/18,6/29/18,6/30/18,7/1/18,7/2/18,7/3/18,7/4/18,7/5/18,7/6/18,7/7/18,7/8/18,7/9/18,7/10/18,7/11/18,7/12/18,7/13/18,7/14/18,7/15/18,7/16/18,7/17/18,7/18/18,7/19/18,7/20/18,...,1/30/20,1/31/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/8/20,2/9/20,2/12/20,2/13/20,2/15/20,2/16/20,2/17/20,2/18/20,2/21/20,2/24/20,2/29/20,3/1/20,3/2/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20
Pranav,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Daljit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aayush,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Yash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Shivam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Yatin Gupta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Himanshu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Amit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Tanya,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [162]:
for index, row in df_by_date_author.iterrows():
  #print(row['Date'],row['Author'],row['Messages'])
  df_structure.set_value(row['Author'],row['Date'],row['Messages'])

df_structure.fillna(0, inplace=True)
df_structure

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  This is separate from the ipykernel package so we can avoid doing imports until


,6/10/18,6/11/18,6/13/18,6/14/18,6/15/18,6/16/18,6/17/18,6/18/18,6/19/18,6/20/18,6/21/18,6/22/18,6/23/18,6/24/18,6/25/18,6/26/18,6/27/18,6/28/18,6/29/18,6/30/18,7/1/18,7/2/18,7/3/18,7/4/18,7/5/18,7/6/18,7/7/18,7/8/18,7/9/18,7/10/18,7/11/18,7/12/18,7/13/18,7/14/18,7/15/18,7/16/18,7/17/18,7/18/18,7/19/18,7/20/18,...,1/30/20,1/31/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/8/20,2/9/20,2/12/20,2/13/20,2/15/20,2/16/20,2/17/20,2/18/20,2/21/20,2/24/20,2/29/20,3/1/20,3/2/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20
Pranav,3,8,31,18,11,14,0,33,25,20,0,11,26,1,5,13,0,0,6,53,37,2,4,13,0,6,5,8,33,14,3,3,5,49,0,1,4,1,4,0,...,3,1,4,3,6,7,3,0,0,36,1,25,15,0,11,2,1,2,9,4,9,21,7,15,8,1,0,5,6,6,31,27,12,35,29,16,10,9,6,2
Daljit,1,7,20,10,57,145,60,10,7,7,5,19,91,7,52,21,0,2,7,85,111,26,25,0,0,24,10,1,75,64,14,7,32,99,2,0,72,2,53,6,...,2,0,4,0,11,6,3,0,20,43,1,26,9,1,0,6,0,0,9,11,1,58,7,7,5,1,9,9,14,1,18,9,48,26,27,25,16,24,56,0
Aayush,1,0,9,2,37,39,11,1,1,0,1,10,12,0,3,3,0,8,1,51,52,1,31,4,6,1,3,4,50,2,0,0,0,3,1,0,1,1,6,0,...,0,0,2,0,8,2,3,1,9,10,2,19,11,1,3,1,5,1,6,21,6,35,6,5,12,1,10,3,21,5,12,28,8,9,18,9,5,6,2,0
Yash,0,0,22,3,2,8,0,68,43,4,0,17,89,6,19,11,0,0,0,41,91,40,10,71,1,38,0,1,38,17,27,3,8,0,0,0,79,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,1,0,2,3,0,3,0,2,0,11,3,2,4,1,2,30,0
Shivam,0,0,3,2,0,5,1,17,1,13,0,0,36,1,1,2,0,4,2,0,0,3,0,32,0,1,0,1,1,3,0,3,0,23,0,0,0,1,4,0,...,1,4,6,0,36,1,15,4,15,30,0,23,3,0,0,7,3,0,13,30,0,4,1,0,0,0,0,27,12,19,12,64,39,43,17,24,20,34,19,2
Yatin Gupta,0,0,27,4,8,63,33,8,1,0,0,4,33,0,32,1,0,3,0,14,54,1,5,25,4,4,4,0,41,25,2,1,8,23,1,3,2,0,15,1,...,1,0,3,1,13,1,1,2,0,15,0,18,1,0,2,5,1,0,5,14,1,15,1,3,3,1,0,0,4,16,21,31,27,16,1,5,3,6,6,0
Himanshu,0,0,2,2,4,64,31,0,2,0,2,6,0,0,6,1,0,2,0,45,52,6,16,0,0,9,0,0,37,13,7,0,0,21,4,1,9,1,7,1,...,0,0,1,0,21,18,10,0,2,8,1,22,4,1,4,0,2,0,5,14,3,17,11,3,16,3,4,6,25,6,22,33,19,20,12,16,11,16,35,0
Amit,0,0,0,2,0,71,19,0,2,0,0,3,0,0,8,0,0,0,0,11,1,0,14,13,0,0,0,0,0,9,0,0,8,0,0,0,0,0,11,0,...,0,0,1,0,4,10,1,0,8,4,1,7,5,1,7,0,1,0,4,16,3,15,1,2,0,6,2,11,13,10,18,3,1,0,0,0,0,14,3,0
NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Tanya,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
writer = pd.ExcelWriter('date_author.xlsx')
df_structure.to_excel(writer)
writer.save()
files.download('date_author.xlsx')

